In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from reservoirpy import ESN
import random
import psutil, time

# Shuffle the entire DataFrame before splitting
df = pd.read_csv('SWaT.csv')
df = df.sample(frac=1, random_state=random.randint(1, 10000)).reset_index(drop=True)

In [11]:
df.columns = df.columns.str.strip()
df['Normal/Attack'] = (df['Normal/Attack'] == 'Attack').astype(int)
X_raw = df.drop(['Timestamp', 'Normal/Attack'], axis=1)
y = df['Normal/Attack']

scaler = MinMaxScaler()
X_raw = pd.DataFrame(scaler.fit_transform(X_raw), columns=X_raw.columns)
X_train, X_test, y_train, y_test = train_test_split(
    X_raw, y, test_size=0.2, stratify=y, random_state=random.randint(1, 10000)
)

In [12]:
corr_matrix = X_train.corrwith(y_train).abs()
top_10_corr = corr_matrix.nlargest(10).index
X_train_corr = X_train[top_10_corr]
X_test_corr = X_test[top_10_corr]
# Remove constant/duplicate columns
X_train_clean = X_train_corr.loc[:, X_train_corr.std() > 0]
X_test_clean = X_test_corr[X_train_clean.columns]
X_train_clean = X_train_clean.T.drop_duplicates().T
X_test_clean = X_test_clean[X_train_clean.columns]
y_train_arr = np.array(y_train.values).reshape(-1, 1)

C:\Users\chand\AppData\Roaming\Python\Python313\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\chand\AppData\Roaming\Python\Python313\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [13]:
from reservoirpy import ESN
from sklearn.metrics import accuracy_score, f1_score

results = []
fixed_spectral_radius = 0.1
fixed_input_scaling = 1.0

# Try only reservoir sizes (10, 20, 30, 40, 50)
for res_size in range(10, 51, 10):
    esn = ESN(
        units=res_size,
        sr=fixed_spectral_radius,
        input_scaling=fixed_input_scaling,
        ridge=1e-6,
        seed=42
    )
    esn = esn.fit(X_train_clean.values, y_train_arr)
    y_pred_test = esn.run(X_test_clean.values)
    y_pred_test = (y_pred_test > 0.5).astype(int).ravel()
    acc = accuracy_score(y_test.values, y_pred_test)
    f1 = f1_score(y_test.values, y_pred_test, average='weighted')
    print(f"Reservoir={res_size} | Accuracy: {acc:.4f}, F1: {f1:.4f}")
    results.append((res_size, acc, f1))

Reservoir=10 | Accuracy: 0.9522, F1: 0.9471
Reservoir=20 | Accuracy: 0.9522, F1: 0.9471
Reservoir=30 | Accuracy: 0.9525, F1: 0.9475
Reservoir=40 | Accuracy: 0.9534, F1: 0.9485
Reservoir=50 | Accuracy: 0.9534, F1: 0.9486


In [14]:
results = []
fixed_spectral_radius = 0.1
fixed_input_scaling = 1.0
for res_size in range(10, 51, 10):
    esn = ESN(units=res_size, sr=fixed_spectral_radius, input_scaling=fixed_input_scaling, ridge=1e-6, seed=42)
    esn = esn.fit(X_train_clean.values, y_train_arr)
    y_pred_test = esn.run(X_test_clean.values)
    y_pred_test = (y_pred_test > 0.5).astype(int).ravel()
    acc = accuracy_score(y_test.values, y_pred_test)
    f1 = f1_score(y_test.values, y_pred_test, average='weighted')
    print(f"Reservoir={res_size} | Acc: {acc:.4f} | F1: {f1:.4f}")
    results.append((res_size, acc, f1))

Reservoir=10 | Acc: 0.9522 | F1: 0.9471
Reservoir=20 | Acc: 0.9522 | F1: 0.9471
Reservoir=30 | Acc: 0.9525 | F1: 0.9475
Reservoir=40 | Acc: 0.9534 | F1: 0.9485
Reservoir=50 | Acc: 0.9534 | F1: 0.9486


In [15]:
results = []
fixed_reservoir = 10  # Or best value found
fixed_input_scaling = 1.0
spectral_radii = [0.1, 0.29, 0.7, 0.8, 1.0]
for spec_rad in spectral_radii:
    esn = ESN(units=fixed_reservoir, sr=spec_rad, input_scaling=fixed_input_scaling, ridge=1e-6, seed=42)
    esn = esn.fit(X_train_clean.values, y_train_arr)
    y_pred_test = esn.run(X_test_clean.values)
    y_pred_test = (y_pred_test > 0.5).astype(int).ravel()
    acc = accuracy_score(y_test.values, y_pred_test)
    f1 = f1_score(y_test.values, y_pred_test, average='weighted')
    print(f"Spectral Radius={spec_rad} | Acc: {acc:.4f} | F1: {f1:.4f}")
    results.append((spec_rad, acc, f1))

Spectral Radius=0.1 | Acc: 0.9522 | F1: 0.9471
Spectral Radius=0.29 | Acc: 0.9513 | F1: 0.9460
Spectral Radius=0.7 | Acc: 0.9513 | F1: 0.9459
Spectral Radius=0.8 | Acc: 0.9513 | F1: 0.9459
Spectral Radius=1.0 | Acc: 0.9513 | F1: 0.9459


In [16]:
results = []
fixed_reservoir = 10
fixed_spectral_radius = 0.1
input_scalings = [0.01, 0.1, 0.5, 0.58, 1.0]
for in_scale in input_scalings:
    esn = ESN(units=fixed_reservoir, sr=fixed_spectral_radius, input_scaling=in_scale, ridge=1e-6, seed=42)
    esn = esn.fit(X_train_clean.values, y_train_arr)
    y_pred_test = esn.run(X_test_clean.values)
    y_pred_test = (y_pred_test > 0.5).astype(int).ravel()
    acc = accuracy_score(y_test.values, y_pred_test)
    f1 = f1_score(y_test.values, y_pred_test, average='weighted')
    print(f"Input Scaling={in_scale} | Acc: {acc:.4f} | F1: {f1:.4f}")
    results.append((in_scale, acc, f1))

Input Scaling=0.01 | Acc: 0.9522 | F1: 0.9470
Input Scaling=0.1 | Acc: 0.9522 | F1: 0.9470
Input Scaling=0.5 | Acc: 0.9522 | F1: 0.9470
Input Scaling=0.58 | Acc: 0.9522 | F1: 0.9471
Input Scaling=1.0 | Acc: 0.9522 | F1: 0.9471
